In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from mlflow import MlflowClient
import seaborn
from sklearn.feature_selection import SelectFromModel
import matplotlib.pyplot as plt
from mlflow.models import infer_signature
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, matthews_corrcoef
# from utils import convert_bool_to_int
from sklearn.model_selection import train_test_split# Text pre-processing
import tensorflow as tf
from tensorflow.python.keras.callbacks import EarlyStopping# Modeling
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
from sklearn.svm import SVC
import ast
from sklearn.model_selection import train_test_split# Text pre-processing
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping# Modeling
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout, GlobalAveragePooling1D, Flatten, SpatialDropout1D, Bidirectional, RepeatVector
import utils
import tqdm
from sklearn import preprocessing
from sklearn.model_selection import KFold
from fast_ml.model_development import train_valid_test_split
from wordsegment import load, segment
from sklearn.model_selection import cross_val_score, cross_validate
import yake
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize

2024-01-14 12:42:39.465219: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-14 12:42:39.980773: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-14 12:42:39.983591: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-14 12:42:41.330076: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
def clean_block_list_new():
    phish = pd.read_csv(filepath_or_buffer='/home/mha/PycharmProjects/thesis/phish_new_jan.csv', sep=',',
                        header=0)
    phish['verification_time'] = pd.to_datetime(phish['verification_time'])
    phish = phish[phish['verification_time'].dt.year == 2023]
    phish[['protocol', 'urlNoProto']] = phish['url'].str.split('://', expand=True, n=1)
    phish = phish.drop(phish[phish.protocol == "http"].index)
    phish.reset_index(inplace=True)
    phish[['fullDomain', 'path']] = phish['urlNoProto'].str.split('/', expand=True, n=1)
    phish = phish.drop(phish[phish['fullDomain'].str.endswith('google.com')].index)
    phish = phish.drop(phish[phish['fullDomain'].str.endswith('amazonaws.com')].index)
    phish = phish.drop(phish[phish['fullDomain'].str.endswith('godaddysites.com')].index)
    phish = phish.drop(phish[phish['fullDomain'].str.endswith('weeblysite.com')].index)
    phish = phish.drop(phish[phish['fullDomain'].str.endswith('mybluehost.me')].index)
    phish = phish.drop(phish[phish['fullDomain'].str.endswith('inmotionhosting.com')].index)
    phish = phish.drop(phish[phish['fullDomain'].str.endswith('firebaseapp.com')].index)
    phish = phish.drop(phish[phish['fullDomain'].str.endswith('googleapis.com')].index)
    phish = phish.drop(phish[phish['fullDomain'].str.endswith('azurefd.net')].index)
    phish.reset_index(inplace=True)
    phish['fullDomain'] = phish['fullDomain'].str.replace('www.', '')
    return phish


def clean_block_list():
    phish = pd.read_csv(filepath_or_buffer='/home/mha/PycharmProjects/thesis/bigQueryCode/phishtank.csv', sep=',',
                        header=0)
    phish['verification_time'] = pd.to_datetime(phish['verification_time'])
    phish = phish[phish['verification_time'].dt.year == 2023]
    phish[['protocol', 'urlNoProto']] = phish['url'].str.split('://', expand=True, n=1)
    phish = phish.drop(phish[phish.protocol == "http"].index)
    phish.reset_index(inplace=True)
    phish[['fullDomain', 'path']] = phish['urlNoProto'].str.split('/', expand=True, n=1)
    phish = phish.drop(phish[phish['fullDomain'].str.endswith('google.com')].index)
    phish = phish.drop(phish[phish['fullDomain'].str.endswith('amazonaws.com')].index)
    phish = phish.drop(phish[phish['fullDomain'].str.endswith('godaddysites.com')].index)
    phish = phish.drop(phish[phish['fullDomain'].str.endswith('weeblysite.com')].index)
    phish = phish.drop(phish[phish['fullDomain'].str.endswith('mybluehost.me')].index)
    phish = phish.drop(phish[phish['fullDomain'].str.endswith('inmotionhosting.com')].index)
    phish = phish.drop(phish[phish['fullDomain'].str.endswith('firebaseapp.com')].index)
    phish = phish.drop(phish[phish['fullDomain'].str.endswith('googleapis.com')].index)
    phish = phish.drop(phish[phish['fullDomain'].str.endswith('azurefd.net')].index)
    phish.reset_index(inplace=True)
    phish['fullDomain'] = phish['fullDomain'].str.replace('www.', '')
    return phish

In [5]:
phish_old = clean_block_list()
phish_new = clean_block_list_new()

In [6]:
phish = [element for element in phish_new.fullDomain if element not in phish_old.fullDomain]

In [8]:
phish_new

,level_0,index,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target,protocol,urlNoProto,fullDomain,path
0,0,2677,8408350,https://secureatttttttttt.weebly.com/,http://www.phishtank.com/phish_detail.php?phis...,2023-12-31T23:42:38+00:00,yes,2023-12-31 23:52:59+00:00,yes,Other,https,secureatttttttttt.weebly.com/,secureatttttttttt.weebly.com,
1,1,2678,8408340,https://conf-bn-cr027.webcindario.com/,http://www.phishtank.com/phish_detail.php?phis...,2023-12-31T22:36:14+00:00,yes,2023-12-31 22:42:24+00:00,yes,Other,https,conf-bn-cr027.webcindario.com/,conf-bn-cr027.webcindario.com,
2,2,2679,8408339,https://new.express.adobe.com/webpage/qP47Wysj...,http://www.phishtank.com/phish_detail.php?phis...,2023-12-31T22:35:18+00:00,yes,2023-12-31 22:42:24+00:00,yes,Other,https,new.express.adobe.com/webpage/qP47Wysjr2uf1,new.express.adobe.com,webpage/qP47Wysjr2uf1
3,3,2680,8408338,https://auhlbrncs.info/1704061867/adls/ls,http://www.phishtank.com/phish_detail.php?phis...,2023-12-31T22:31:25+00:00,yes,2023-12-31 22:42:24+00:00,yes,Other,https,auhlbrncs.info/1704061867/adls/ls,auhlbrncs.info,1704061867/adls/ls
4,4,2681,8408329,https://cloudflare-ipfs.com/ipfs/QmQ6TaMx2irPu...,http://www.phishtank.com/phish_detail.php?phis...,2023-12-31T21:01:55+00:00,yes,2023-12-31 21:03:56+00:00,yes,Other,https,cloudflare-ipfs.com/ipfs/QmQ6TaMx2irPurLzd4eN4...,cloudflare-ipfs.com,ipfs/QmQ6TaMx2irPurLzd4eN4CjawGvkgAZj4NK2D355z...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25141,31845,38933,7529455,https://anchorprotocol.mypixieset.com/,http://www.phishtank.com/phish_detail.php?phis...,2022-05-30T17:17:34+00:00,yes,2023-01-09 03:39:52+00:00,yes,Other,https,anchorprotocol.mypixieset.com/,anchorprotocol.mypixieset.com,
25142,31846,38940,7526004,https://traderjoexyzcom.pilotpoc.com/,http://www.phishtank.com/phish_detail.php?phis...,2022-05-25T17:47:17+00:00,yes,2023-04-14 01:56:03+00:00,yes,Other,https,traderjoexyzcom.pilotpoc.com/,traderjoexyzcom.pilotpoc.com,
25143,31848,38945,7525114,https://zonasegura.mibanco.com.pe/frontend/login/,http://www.phishtank.com/phish_detail.php?phis...,2022-05-24T22:58:10+00:00,yes,2023-01-09 03:32:10+00:00,yes,Other,https,zonasegura.mibanco.com.pe/frontend/login/,zonasegura.mibanco.com.pe,frontend/login/
25144,31850,38953,7523669,https://www.sutori.com/en/story/bt-broadband-b...,http://www.phishtank.com/phish_detail.php?phis...,2022-05-23T07:46:45+00:00,yes,2023-07-31 01:05:54+00:00,yes,Other,https,www.sutori.com/en/story/bt-broadband-bill--TrM...,sutori.com,en/story/bt-broadband-bill--TrMPqS29QtKK3XWNDU...


In [10]:
sentences = []
load()
for domain in phish_new.fullDomain:
    domain_noTLD = domain.rsplit(".", 1)[0]
    parts = domain_noTLD.split(".")
    sentence = ""
    for part in parts:
        segments = segment(part)
        bit = " ".join(segments)
        sentence = sentence + " " + bit
    sentences.append(sentence)
phish_new["domain_sent_new"] = sentences

In [15]:
phish_new

,level_0,index,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target,protocol,urlNoProto,fullDomain,path,domain_sent_new
0,0,2677,8408350,https://secureatttttttttt.weebly.com/,http://www.phishtank.com/phish_detail.php?phis...,2023-12-31T23:42:38+00:00,yes,2023-12-31 23:52:59+00:00,yes,Other,https,secureatttttttttt.weebly.com/,secureatttttttttt.weebly.com,,secure att tttt tttt weebl y
1,1,2678,8408340,https://conf-bn-cr027.webcindario.com/,http://www.phishtank.com/phish_detail.php?phis...,2023-12-31T22:36:14+00:00,yes,2023-12-31 22:42:24+00:00,yes,Other,https,conf-bn-cr027.webcindario.com/,conf-bn-cr027.webcindario.com,,confbncr027 webcindario
2,2,2679,8408339,https://new.express.adobe.com/webpage/qP47Wysj...,http://www.phishtank.com/phish_detail.php?phis...,2023-12-31T22:35:18+00:00,yes,2023-12-31 22:42:24+00:00,yes,Other,https,new.express.adobe.com/webpage/qP47Wysjr2uf1,new.express.adobe.com,webpage/qP47Wysjr2uf1,new express adobe
3,3,2680,8408338,https://auhlbrncs.info/1704061867/adls/ls,http://www.phishtank.com/phish_detail.php?phis...,2023-12-31T22:31:25+00:00,yes,2023-12-31 22:42:24+00:00,yes,Other,https,auhlbrncs.info/1704061867/adls/ls,auhlbrncs.info,1704061867/adls/ls,au hlb rncs
4,4,2681,8408329,https://cloudflare-ipfs.com/ipfs/QmQ6TaMx2irPu...,http://www.phishtank.com/phish_detail.php?phis...,2023-12-31T21:01:55+00:00,yes,2023-12-31 21:03:56+00:00,yes,Other,https,cloudflare-ipfs.com/ipfs/QmQ6TaMx2irPurLzd4eN4...,cloudflare-ipfs.com,ipfs/QmQ6TaMx2irPurLzd4eN4CjawGvkgAZj4NK2D355z...,cloud flare i pfs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25141,31845,38933,7529455,https://anchorprotocol.mypixieset.com/,http://www.phishtank.com/phish_detail.php?phis...,2022-05-30T17:17:34+00:00,yes,2023-01-09 03:39:52+00:00,yes,Other,https,anchorprotocol.mypixieset.com/,anchorprotocol.mypixieset.com,,anchor protocol my pixie set
25142,31846,38940,7526004,https://traderjoexyzcom.pilotpoc.com/,http://www.phishtank.com/phish_detail.php?phis...,2022-05-25T17:47:17+00:00,yes,2023-04-14 01:56:03+00:00,yes,Other,https,traderjoexyzcom.pilotpoc.com/,traderjoexyzcom.pilotpoc.com,,trader joe xyz com pilot poc
25143,31848,38945,7525114,https://zonasegura.mibanco.com.pe/frontend/login/,http://www.phishtank.com/phish_detail.php?phis...,2022-05-24T22:58:10+00:00,yes,2023-01-09 03:32:10+00:00,yes,Other,https,zonasegura.mibanco.com.pe/frontend/login/,zonasegura.mibanco.com.pe,frontend/login/,zona segura mi banco com
25144,31850,38953,7523669,https://www.sutori.com/en/story/bt-broadband-b...,http://www.phishtank.com/phish_detail.php?phis...,2022-05-23T07:46:45+00:00,yes,2023-07-31 01:05:54+00:00,yes,Other,https,www.sutori.com/en/story/bt-broadband-bill--TrM...,sutori.com,en/story/bt-broadband-bill--TrMPqS29QtKK3XWNDU...,sutor i


In [11]:
text = ". ".join(phish_new.domain_sent_new)

In [12]:
text

' secure att tttt tttt weebl y.  confbncr027 webcindario.  new express adobe.  au hlb rncs.  cloud flare i pfs.  oea06.  connect chase 23 web.  warning info ads683ef web.  yin eng chem.  single southern guy.  niaz cheh.  jy ga wy.  dlr f.  amiss i a.  adi rcaf.  self referral check.  control soluciones.  d apps debugger.  y market.  informacje sopot.  new secret insta gram blogspot.  wojcik sklep.  dhl track delivery.  us pt gao.  serve usps os.  serve usps cs.  1 netflix.  mysterious polyester reading glitch.  colombini onion weebl y.  meta mx ck logie web flow.  tg teleg arm if.  usps llw.  hindman auctions pages.  psi ko drama merkezi.  blue alligator spex i code any app.  blue alligator spex i code any app.  blue alligator spex i code any app.  edfkjhnedf23w ul craft.  skin seller.  da chikan.  blue alligator spex i code any app.  ppi co.  anka food.  anka food.  innotech technologies.  my own item.  track ng parcel.  login singapor re internet baki ings.  exodsw3bwallett git book.

In [14]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def extract_keywords(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]

    # Convert back to a sentence
    processed_text = ' '.join(lemmatized_tokens)

    # Calculate TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([processed_text])
    feature_names = vectorizer.get_feature_names_out()

    # Get the top keywords
    indices = tfidf_matrix.toarray().argsort(axis=1)[:, ::-1]
    top_n = 60  # Number of top keywords to extract
    top_keywords = [feature_names[idx] for idx in indices.flatten()[:top_n]]

    return top_keywords

# Extract keywords
keywords = extract_keywords(text)
print("Keywords:", keywords)

[nltk_data] Downloading package punkt to /home/mha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/mha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/mha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Keywords: ['pfs', 'cfi', 'cloud', 'flare', 'web', 'eth', 'aragon', 'site', 'r2', 'domain', 'crazy', 'beat', 'worker', 'new', 'express', 'adobe', 'ba', 'storage', 'fy', 'nft', 'page', 'wave', 'com', 'webcindario', 'wix', 'square', 'best', 'practice', 'usps', 'start', 'tinyurl', 'blogspot', 'portfolio', 'help', 'co', 'link', 'code', 'form', 'bit', 'flow', 'app', 'weebl', 'service', 'mail', 'online', 'login', 'keep', 'url', 'gateway', 'post', 'al', 'baf', 'ver', 'office', 'id', 'cel', 'ap', 'mardo', 'box', 'net']


In [15]:
def top_tranco():
    tranco_raw = pd.read_csv(filepath_or_buffer='/home/mha/PycharmProjects/thesis/bigQueryCode/tranco.csv', sep=',',
                             header=0)
    return tranco_raw.drop(tranco_raw[tranco_raw['rank'] > 50000].index)

In [16]:
tranco = top_tranco()

In [17]:
tranco

,rank,domain
0,1,google.com
1,2,facebook.com
2,3,amazonaws.com
3,4,microsoft.com
4,5,apple.com
...,...,...
49995,49996,baby-walz.de
49996,49997,adtrace.io
49997,49998,arena1.com
49998,49999,eo.nl


In [18]:
sentences = []
load()
for domain in tranco.domain:
    domain_noTLD = domain.rsplit(".", 1)[0]
    parts = domain_noTLD.split(".")
    sentence = ""
    for part in parts:
        segments = segment(part)
        bit = " ".join(segments)
        sentence = sentence + " " + bit
    sentences.append(sentence)
tranco["domain_sent"] = sentences

In [19]:
text_tranco = " ".join(tranco.domain_sent)

In [20]:
keywords_tranco = extract_keywords(text_tranco)
print("Keywords:", keywords_tranco)

Keywords: ['com', 'co', 'cdn', 'gov', 'net', 'dns', 'cloud', 'news', 'ad', 'ac', 'edu', 'web', 'go', 'online', 'game', 'blog', 'google', 'app', 'medium', 'host', 'porn', 'tv', 'video', 'time', 'server', 'data', 'bank', 'live', 'org', 'world', 'service', 'sport', 'mail', 'la', 'link', 'tech', 'get', 'api', 'play', 'one', 'lab', 'site', 'free', 'shop', 'hosting', 'bet', 'ip', 'de', 'network', 'hub', 'static', 'pro', 'box', 'uni', 'home', 'plus', 'sex', 'nic', 'post', 'daily']


In [22]:
words_not_in_tranco = [word for word in keywords if word not in keywords_tranco]

In [24]:
len(words_not_in_tranco)

47

In [31]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def extract_keywords(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in text if word.isalnum() and word not in stop_words]

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]

    # Convert back to a sentence
    processed_text = ' '.join(lemmatized_tokens)

    # Calculate TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(processed_text)
    feature_names = vectorizer.get_feature_names_out()

    # Get the top keywords
    indices = tfidf_matrix.toarray().argsort(axis=1)[:, ::-1]
    top_n = 60  # Number of top keywords to extract
    top_keywords = [feature_names[idx] for idx in indices.flatten()[:top_n]]

    return top_keywords

# Example text
# sample_text = "This is an example sentence to demonstrate keyword extraction using Python. Python is a versatile programming language."

# Extract keywords
keywords_phish = extract_keywords(text)
print("Keywords:", keywords_phish)

[nltk_data] Downloading package punkt to /home/mha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/mha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/mha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


ValueError: Iterable over raw text documents expected, string object received.

In [27]:
text

'This is the first sentence. This is the second sentence. And this is the third sentence.'

In [ ]:
from summa import keywords
TR_keywords = keywords.keywords(text, scores=True)
print(TR_keywords[0:10])

In [ ]:
def extract_keywords(text):
    # Tokenize text into sentences
    sentences = sent_tokenize(text)
    
    # Create TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    
    # Fit and transform the sentences
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
    
    # Get feature names (words) and their corresponding TF-IDF scores
    feature_names = tfidf_vectorizer.get_feature_names_out()
    tfidf_scores = tfidf_matrix.toarray()
    
    # Create a list of dictionaries, where each dictionary represents a sentence and its TF-IDF scores
    sentence_keywords = []
    for i, sentence in enumerate(sentences):
        keyword_dict = {feature_names[j]: tfidf_scores[i][j] for j in range(len(feature_names))}
        sentence_keywords.append({'sentence': sentence, 'keywords': keyword_dict})
    
    return sentence_keywords

# Example usage
keywords = extract_keywords(text)

In [32]:
keywords

[{'sentence': 'This is the first sentence.',
  'keywords': {'second': 0.0, 'sentence': 1.0}},
 {'sentence': 'This is the second sentence.',
  'keywords': {'second': 0.8610369959439764, 'sentence': 0.5085423203783267}},
 {'sentence': 'And this is the third sentence.',
  'keywords': {'second': 0.0, 'sentence': 1.0}}]

In [12]:
text_list = list(phish_new.domain_sent_new)

In [14]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(text_list)

In [18]:
feature_names = vectorizer.get_feature_names_out()

# Get the top keywords
indices = X.toarray().argsort(axis=1)[:, ::-1]
top_n = 60  # Number of top keywords to extract
top_keywords = [feature_names[idx] for idx in indices.flatten()[:top_n]]

In [19]:
top_keywords

['tttt',
 'secure',
 'att',
 'weebl',
 'deem',
 'decency',
 'decent',
 'decentralized',
 'decf0t',
 'deck',
 'declaracion',
 'deedfe1',
 'deep',
 'decebal0',
 'deepak',
 'deeppbssniiees674838',
 'deepsky',
 'deer',
 'dees',
 'def',
 'defense',
 'defi',
 'december',
 'zzz8cd',
 'decal',
 'deaton',
 'de3209',
 'de592cd2c63',
 'de827162',
 'de8b',
 'de9028',
 'de9138023',
 'dealer',
 'deals',
 'deb',
 'dei',
 'debi',
 'debit',
 'debra',
 'debre',
 'debt',
 'debug1',
 'debugger',
 'dec3',
 'deg',
 'del',
 'dejmy38utchefilip',
 'denmark',
 'dem',
 'demain',
 'demais',
 'demo',
 'demo2',
 'demo3',
 'demo99',
 'demos',
 'dental',
 'deltas',
 'dentist',
 'denvacarrie46']